In [2]:
import pandas as pd
import json
import sqlite3

# ================================
# STEP 1: LOAD CSV DATA (orders.csv)
# ================================
orders_df = pd.read_csv("orders.csv")

# ================================
# STEP 2: LOAD JSON DATA (users.json)
# ================================
with open("users.json", "r") as f:
    users_data = json.load(f)

if isinstance(users_data, list):
    users_df = pd.DataFrame(users_data)
elif "users" in users_data:
    users_df = pd.DataFrame(users_data["users"])
else:
    users_df = pd.DataFrame([users_data])

# ================================
# STEP 3: LOAD SQL DATA (restaurants.sql)
# ================================
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
table_name = cursor.fetchone()[0]

restaurants_df = pd.read_sql_query(
    f"SELECT * FROM {table_name}", conn
)

# ================================
# STEP 4: MERGE THE DATA (LEFT JOIN)
# ================================
merged_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    merged_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)

# ================================
# STEP 5: CONVERT FINAL DATASET TO CSV
# ================================
file_name = "final_food_delivery_dataset.csv"
final_df.to_csv(file_name, index=False)

# ================================
# DOWNLOAD CSV FILE (FOR COLAB)
# ================================
try:
    from google.colab import files
    files.download(file_name)
except:
    pass

# ================================
# DISPLAY FINAL DATASET DETAILS
# ================================
print("Final Dataset:")
print(final_df)


Final Dataset:
      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                   restaurant_name_x       name       city membership  \
0                  New Foods Chinese  User_2508  Hyderabad    Regular   
1     Ruchi Curry Hous